In [1]:
from aicsimageio import AICSImage
import napari
from aicsimageio.readers import CziReader
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# VISUALIZATION

# visualize the image with napari using its numpy array
def visualize_napari(numpy_img: np.ndarray, name):
    """
    :param numpy_img: image to be visualized
    """
    with napari.gui_qt():
        viewer = napari.Viewer()
        viewer.add_image(numpy_img, name=name)


# visualize multiple images at once
def visualize_all_list_napari(numpy_img_list: np.ndarray, names):
    """
    :param numpy_img_list: list containing different images to be visualized
    """
    with napari.gui_qt():
        viewer = napari.Viewer()
        for i, img in enumerate(numpy_img_list):
            viewer.add_image(img, name=names[i])



# PREPROCESSING

# rescale and convert image 
def rescale(image: np.ndarray):
    """
    :param image: to  be rescaled
    :return: rescaled image
    """
    rescaled_image = (image - np.min(image)) / (np.max(image) - np.min(image)) * 255
    # rescaled_image = np.round(rescaled_image).astype(np.uint16)
    rescaled_image = np.round(rescaled_image).astype('uint8')
    return rescaled_image


# sharpen image
def sharpen(image: np.ndarray):
    """
    Sharpen the image
    :param image: image to be sharpened
    :return: sharp image
    """
    kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    return cv.filter2D(image, -1, kernel)



# POSTPROCESSING 

# add 2D bounding boxes to the image
def add_bounding_boxes(image, stats):
    """
    Add white rectangles around bacilli, based on conected components

    :param image: image with bacilli to be boxed
    :param coordinates:  coordinates of the center of the bacillus
    """
    for i in range(1,len(stats)):
            x = stats[i][0] - 5
            #x_max = coordinates[i][0]
            y = stats[i][1] - 5
            #y_max = coordinates[i][1]
            h=stats[i][3]
            w=stats[i][2]
            cv.rectangle(image, (x, y), (x+w+10, y+h+10), (255, 0, 0), 1)
    
    return image

In [3]:
# Load the image
reader = CziReader("extern_Synlab_2156_17_3_MTB.czi")

# Get whole image
smear = reader.get_image_data("MYX", C=0)

In [20]:
# save 675th tile from smear in separate variable and create a copy for future purposes
img = smear[674]
img_copy = img.copy()

In [77]:
maximum = np.max(img)
minimum = np.min(img)
print(maximum, minimum)

wid = img.shape[1]
hgt = img.shape[0]
print(wid, hgt)

16414 104
1504 2048


In [73]:
rescaled_img = rescale(img)
sharpened_img = sharpen(rescaled_img)

In [74]:
blurred_img = cv.medianBlur(rescaled_img,1)
th2 = cv.adaptiveThreshold(rescaled_img,255,cv.ADAPTIVE_THRESH_MEAN_C,cv.THRESH_BINARY,25,-5) # negative Werte erhöhen den Schwarzanteil, positive den Weißanteil
th3 = cv.adaptiveThreshold(rescaled_img,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,cv.THRESH_BINARY,25,-5)

th2_copy = th2.copy()
th3_copy = th3.copy()

num_labels2, labels_im2, stats2, centroids2 = cv.connectedComponentsWithStats(th2_copy, connectivity=8)
num_labels3, labels_im3, stats3, centroids3 = cv.connectedComponentsWithStats(th3_copy, connectivity=8)

# clean from noise (setting connected components of five or less white pixels to zero)
for i in range(1, num_labels2):
    if stats2[i][4] < 6:
        th2_copy[labels_im2 == i] = 0

for i in range(1, num_labels3):
    if stats3[i][4] < 6:
        th3_copy[labels_im3 == i] = 0

num_labels2, labels_im2, stats2, centroids2 = cv.connectedComponentsWithStats(th2_copy, connectivity=8)
num_labels3, labels_im3, stats3, centroids3 = cv.connectedComponentsWithStats(th3_copy, connectivity=8)

th2_boxes = add_bounding_boxes(th2_copy, stats2)
th3_boxes = add_bounding_boxes(th3_copy, stats3)

# add boxes to original image
img_copy = img.copy()
og_boxes2 = add_bounding_boxes(img_copy, stats2)
img_copy = img.copy()
og_boxes3 = add_bounding_boxes(img_copy, stats3)

titles = ['Original Rescaled Image', 'Sharpened Image', 'Blurred Image', 'Adaptive Mean Thresholding', 'Adaptive Gaussian Thresholding', 'Boxes Mean Thresholding', 'Boxes Gaussian Thresholding', 'Boxes OG Mean', 'Boxes OG Gaussian']
images = [rescaled_img, sharpened_img, blurred_img, th2, th3, th2_boxes, th3_boxes, og_boxes2, og_boxes3]

In [75]:
# visualize
visualize_all_list_napari(images, titles)

c:\Users\idabu\.vscode\TBProject\.venv\lib\site-packages\napari\_qt\qt_event_loop.py:284: FutureWarning: 
The 'gui_qt()' context manager is deprecated.
If you are running napari from a script, please use 'napari.run()' as follows:

    import napari

    viewer = napari.Viewer()  # no prior setup needed
    # other code using the viewer...
    napari.run()

In IPython or Jupyter, 'napari.run()' is not necessary. napari will automatically
start an interactive event loop for you: 

    import napari
    viewer = napari.Viewer()  # that's it!

  warn(
